In [158]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd

In [159]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# device="cpu"
print(f"Using {device} device")

Using cuda device


In [160]:
import json

data=[]

# data_file = './labels.json'
data_file = './full_dataset/converted.json'

using = "pandas"


if(using=="json"):
    # Load JSON data from the file
    with open(data_file) as f:
        data = json.load(f)
    print(data[1]['time'])
else:
    data =pd.read_json(data_file)
    print(data.head())

          time                                              chair  \
0  00:00.00000  [[-0.1140332378, -1.0363261758, 0.0844578892, ...   
1  00:00.08103  [[-0.1087164686, -1.0356720574, 0.0847318111, ...   
2  00:00.15946  [[-0.1033996995, -1.035017939, 0.085005733, 1....   
3  00:00.23964  [[-0.09808293030000001, -1.0343638205, 0.08527...   
4  00:00.31915  [[-0.09276616110000001, -1.0337097021, 0.08555...   

                                               stool  \
0  [[2.1505855001, -1.4536903501, 0.1369566768, 1...   
1  [[2.1473772297, -1.4468101854, 0.1367251761, 1...   
2  [[2.1441689593, -1.4399300207, 0.1364936754000...   
3  [[2.1409606889, -1.4330498561, 0.1362621746, 1...   
4  [[2.1377524185, -1.4261696914, 0.1360306739, 1...   

                                               table  \
0  [[0.1156135214, -2.2455869696, 0.1178784221, 1...   
1  [[0.1152638103, -2.2443210638, 0.117856133, 1....   
2  [[0.11491409920000001, -2.243055158, 0.1178338...   
3  [[0.114564388, -2.241

In [161]:
#Remove Mis-aligend Files
for item in ["chair","stool","table","wall"]:
    for index,i in enumerate(data[item]):
        if type(i)== dict:
            data=data.drop(index)

In [162]:
for item in ["chair","stool","table","wall"]:
    for index,i in enumerate(data[item]):
        if type(i)== dict:
            print(index)

In [163]:
import open3d as o3d
import numpy as np

def readPCDFile(fileName):
    fileName =fileName.replace(":","_")
#     pcd_path = f"data_set/{fileName}.pcd"
    pcd_path = f"full_dataset/pcd/{fileName}.pcd"
    point_cloud = o3d.io.read_point_cloud(pcd_path)
    point_cloud = point_cloud.voxel_down_sample(voxel_size=0.086)
    output=torch.from_numpy(np.asanyarray(point_cloud.points)).float()
    output=torch.nn.functional.pad(output, (0, 0, 0, 1600 - output.size(0)))
    output = output.to(device=device)
    return output
readPCDFile("00:00.08103").shape

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [142]:
class CustomDataset(Dataset):
    
    def __init__(self, data):
        self.labels=data

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx] if using=="json" else self.labels.iloc[idx]
        point_cloud_data = readPCDFile(label['time'])
        label_data=torch.tensor(label['chair'],device=device)
        return [point_cloud_data, label_data]

In [143]:
dataset = CustomDataset(data=data)
dataset_loader = DataLoader(dataset, batch_size=1, shuffle=True)

In [144]:
dataset_loader.dataset[0]

[tensor([[ 32.1687, -18.3864,   3.2417],
         [ -0.5897,   2.7161,   0.2432],
         [ -0.1999,   3.2694,   0.2866],
         ...,
         [  0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000]], device='cuda:0'),
 tensor([[-0.1140, -1.0363,  0.0845,  1.8055,  1.1400,  0.4269,  0.0000,  0.0000,
           0.0000],
         [ 1.2384, -3.4363,  0.1242,  1.2163,  1.3069,  0.7050,  0.0000,  0.0000,
           0.0000],
         [ 3.2114, -3.2485,  0.1857,  0.9632,  1.1087,  0.9266,  0.0000,  0.0000,
          48.5323],
         [ 5.4827,  0.1195,  0.1603,  1.7691,  1.9000,  0.9465,  0.0000,  0.0000,
           0.0000]], device='cuda:0')]

In [145]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.model=nn.Sequential(
#           # Define convolutional layers
            nn.ReLU(),
            nn.Conv1d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
            nn.MaxPool1d(kernel_size=16, stride=2),
            nn.Conv1d(in_channels=16, out_channels=16, kernel_size=4, padding=1),
            nn.Linear(792,16),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(16,1),
            nn.Flatten(),
        )
        self.prob=nn.Sequential(
            nn.ReLU(),
            nn.Linear(16,32),
            nn.Linear(32,10),
            nn.Softmax(dim=1)
        )
        self.boxes=nn.Sequential(
            nn.ReLU(),
            nn.Linear(16,32),
            nn.Linear(32,90)
        )
       
    def forward(self,x):
        
        base = self.model(x)
        _base = base.clone()
        prob = self.prob(base)
        boxes = self.boxes(_base).view(-1, 10,9)
        return prob,boxes
        

In [156]:


model = NeuralNetwork().to(device)
print(model)

criterion = nn.MSELoss()
criterion_prob = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.001)



RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [157]:
import matplotlib.pyplot as plt


def pad_labels(label, max_len=10):
    if len(label[0])>0 :
        pad_width=max_len-len(label[0])
        padded_label = torch.nn.functional.pad(label, (0, 0, 0,pad_width,0,0))
        return padded_label
    else:
        return torch.zeros(1,10,9,device=device)

loss_values = []
accuracy_values = []

for epoch in range(100):
    running_loss = 0.0
    total_correct = 0
    total_samples = 0
    
    for i, _data in enumerate(dataset_loader, 0):
        if(i>15):
            break
        try:
            inputs, labels = _data
            optimizer.zero_grad()
            prob, boxes = model(inputs.transpose(1, 2))
            loss = criterion(boxes, pad_labels(labels.float()))
#             prob_test = np.zeros((1, 10))
#             prob_test[0, :len(labels[0])] = 1 / (len(labels[0]) if len(labels[0])>0 else 1 )
#             loss_prob = criterion_prob(prob, torch.tensor(prob_test, device=device))
#             total_loss =  loss_prob
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        except:
            print("Exception Occured at "+i)
            pass
        # Update statistics
       
        
    if epoch % 100 == 0:  # Print every 10 mini-batches
        print('[%d / 100] loss: %.3f' % (epoch + 1, running_loss / 100), end="\r", flush=True)
#             running_loss = 0.0

    loss_values.append(running_loss/100)

# Plot the loss function
plt.plot(loss_values, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()
plt.show()


print('Finished Training')


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [134]:
with torch.no_grad():
    prediction = model(readPCDFile("00:07.99279").unsqueeze(0).transpose(1,2))
    print(prediction)
# print(readPCDFile("00:07.99279").unsqueeze(0).shape)

(tensor([[5.4256e-01, 4.5744e-01, 7.8974e-39, 1.4728e-42, 1.4013e-45, 8.3489e-42,
         1.5204e-42, 4.3277e-38, 1.2766e-40, 3.4013e-40]], device='cuda:0'), tensor([[[ 5.5844e-01, -5.4920e-01,  3.4757e-01,  1.6946e+00,  1.0740e+00,
           1.0163e+00,  1.1024e-01, -4.3087e-02,  9.6505e+01],
         [ 1.5524e+00, -1.2016e+00,  2.3682e-01,  5.9967e-01,  8.9543e-01,
          -5.3809e-02, -1.3164e-01, -4.0612e-01,  1.1910e+02],
         [ 5.6126e-01,  1.9396e+00, -2.3728e-02,  1.2576e+00,  1.1388e+00,
           1.1933e+00,  3.2872e-01,  3.2016e-01, -1.0077e+01],
         [-8.3124e-01,  1.0617e-01,  4.0135e-01,  4.2199e-01, -2.0902e-01,
           6.3450e-01, -8.6333e-03,  5.9295e-02,  9.0797e-02],
         [ 4.3726e-02,  2.3291e-01, -4.1910e-01,  1.2162e-01,  6.1911e-02,
           6.1448e-01, -2.0163e-01, -4.9227e-01,  3.3331e-02],
         [ 6.4744e-02, -3.7859e-01,  2.5120e-01, -4.2707e-01, -1.0397e-01,
          -1.1497e-01,  1.2403e-02,  1.7447e-02,  1.9050e-01],
         [-5.

In [206]:
m = nn.Conv1d(128,16,kernel_size=3)
input = torch.randn(128,3)
output = m(input)
output.shape

torch.Size([16, 1])

In [219]:
import torch
import torch.nn as nn

class LiDARConvNet(nn.Module):
    def __init__(self):
        super(LiDARConvNet, self).__init__()
        
        # Define convolutional layers
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=16, kernel_size=3)
#         self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
#         self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        
#         # Define max pooling layer
#         self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
#         # Define fully connected layers
#         self.fc1 = nn.Linear(64 * 365, 128)  # Adjusted based on the expected size after pooling
#         self.fc2 = nn.Linear(128, 64)
#         self.fc3 = nn.Linear(64, 10)  # Example output size
        
#         # Define activation function
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # Apply convolutional and pooling layers
        x = self.relu(self.conv1(x))
#         x = self.pool(x)
#         x = self.relu(self.conv2(x))
#         x = self.pool(x)
#         x = self.relu(self.conv3(x))
#         x = self.pool(x)
        
#         # Flatten the output for fully connected layers
#         x = x.view(-1, 64 * 365)
        
#         # Apply fully connected layers
#         x = self.relu(self.fc1(x))
#         x = self.relu(self.fc2(x))
#         x = self.fc3(x)
        
        return x

# Example usage
lidar_data = torch.randn( 1460, 3)  # Assuming you have 1460 points in your LiDAR point cloud
# conv_net = LiDARConvNet()
# output = conv_net(lidar_data.unsqueeze(1).transpose(1, 2))  # Transpose to (batch_size, channels, sequence_length)
print(lidar_data.unsqueeze(1).shape,lidar_data.unsqueeze(1).transpose(1,2).shape)  # Output shape would be (batch_size, num_classes)


torch.Size([1460, 1, 3]) torch.Size([1460, 3, 1])


In [150]:
a=np.array([0.0822, 0.0629, 0.1309, 0.0699, 0.1266, 0.0716, 0.1137, 0.0919, 0.1356,
         0.1148])
a.sum()

1.0001

In [151]:
a>0.05

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [177]:
arr = np.zeros((1, 10))
arr[0, :4] = 1 / 4

In [241]:
target = torch.tensor(np.ones((1,10)))
torch.sum(target-arr*4)

tensor(6., dtype=torch.float64)

In [63]:
prob_test = np.zeros((1, 10))
prob_test[0, :0] = 1 / 1

In [64]:
prob_test

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [114]:
d = {'A':[1,2,4,6],"B":[1,2,4,5]}
df =pd.DataFrame(d)

In [116]:
df.drop(1)

,A,B
0,1,1
2,4,4
3,6,5


In [117]:
df

,A,B
0,1,1
1,2,2
2,4,4
3,6,5
